In [3]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Load CSV files
df_one = pd.read_csv("/home/harald_stabbetorp/Master/e-prop_master_nmbu/sound_fet/one_encoded_features.csv")  # Shape: (num_samples, 12800)
df_two = pd.read_csv("/home/harald_stabbetorp/Master/e-prop_master_nmbu/sound_fet/two_encoded_features.csv")  # Shape: (num_samples, 12800)
df_four = pd.read_csv("/home/harald_stabbetorp/Master/e-prop_master_nmbu/sound_fet/four_encoded_features.csv")  # Shape: (num_samples, 12800)
df_six = pd.read_csv("/home/harald_stabbetorp/Master/e-prop_master_nmbu/sound_fet/six_encoded_features.csv")  # Shape: (num_samples, 12800)
df_silence = pd.read_csv("/home/harald_stabbetorp/Master/e-prop_master_nmbu/sound_fet/zeros.csv")  # Shape: (num_samples, 12800)

# Reshape to (num_samples, 25, 512)
X_one = df_one.values.reshape(-1, 25, 512)
X_two = df_two.values.reshape(-1, 25, 512)
X_four = df_four.values.reshape(-1, 25, 512)
X_six = df_six.values.reshape(-1, 25, 512)
X_silence = df_silence.values.reshape(-1, 25, 512)

# Define labels: Silence=0, "one"=1, "two"=2
y_silence = np.zeros((X_silence.shape[0], 1))
y_one = np.ones((X_one.shape[0], 1))  # Label "one" as 1
y_two = np.full((X_two.shape[0], 1), 2)  # Label "two" as 2
y_four = np.full((X_four.shape[0], 1), 3)  # Label "four" as 3
y_six = np.full((X_six.shape[0], 1), 4)  # Label "six" as 4

# Concatenate sequences in alternating order
X_combined = np.vstack((X_silence, X_one, X_silence, X_two, X_silence, X_four, X_silence, X_six))
y_combined = np.vstack((y_silence, y_one, y_silence, y_two, y_silence, y_four, y_silence, y_six))

# Repeat the sequence 10 times to make the dataset longer
X_combined = np.tile(X_combined, (10, 1, 1))  # Repeat X_combined 10 times
y_combined = np.tile(y_combined, (10, 1))     # Repeat y_combined 10 times

# Shuffle the dataset (important)
X_combined, y_combined = shuffle(X_combined, y_combined, random_state=42)

# Split into training and testing (using the same data for both)
X_train, X_test = X_combined, X_combined
y_train, y_test = y_combined, y_combined

print("X_train shape:", X_train.shape)  # (total_samples * 10, 25, 512)
print("y_train shape:", y_train.shape)  # (total_samples * 10, 1)


X_train shape: (80, 25, 512)
y_train shape: (80, 1)


In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Define model
inputs = Input(shape=(25, 512))
x = LSTM(256, return_sequences=False)(inputs)
x = Dense(128, activation='relu')(x)
x = Dense(5, activation='softmax')(x)  # 3 output classes: Silence, One, Two

model = Model(inputs, x)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)


Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 897ms/step - accuracy: 0.5417 - loss: 1.6073 - val_accuracy: 1.0000 - val_loss: 1.3246
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - accuracy: 1.0000 - loss: 1.3090 - val_accuracy: 1.0000 - val_loss: 0.9982
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step - accuracy: 1.0000 - loss: 1.1367 - val_accuracy: 1.0000 - val_loss: 0.8078
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step - accuracy: 1.0000 - loss: 0.9472 - val_accuracy: 1.0000 - val_loss: 0.6934
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step - accuracy: 1.0000 - loss: 0.8152 - val_accuracy: 1.0000 - val_loss: 0.6111
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step - accuracy: 1.0000 - loss: 0.8262 - val_accuracy: 1.0000 - val_loss: 0.5508
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step - accuracy: 1.0000 - loss: 0.7801 - val_accuracy: 1.0000 - val_loss: 0.4740
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step - accuracy: 1.0000 - loss: 0.6168 - val_accuracy: 1.0000 - val_loss:

In [5]:
# Evaluate on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

# Make predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert softmax output to class index

print("Predicted Classes:", y_pred_classes)  # Show first 10 predictions
print("True Classes:", y_test.flatten())  # Show first 10 actual labels

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 2.6481e-04
Test Accuracy: 1.0000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
Predicted Classes: [0 0 0 4 0 0 0 0 0 0 1 1 2 2 0 3 1 3 4 0 1 0 3 4 0 0 0 0 0 4 0 4 4 0 3 2 0
 1 0 3 0 2 1 0 0 1 0 0 0 0 0 0 4 2 1 0 0 0 2 3 1 2 0 2 2 4 3 3 3 1 0 3 0 4
 0 0 0 4 0 2]
True Classes: [0. 0. 0. 4. 0. 0. 0. 0. 0. 0. 1. 1. 2. 2. 0. 3. 1. 3. 4. 0. 1. 0. 3. 4.
 0. 0. 0. 0. 0. 4. 0. 4. 4. 0. 3. 2. 0. 1. 0. 3. 0. 2. 1. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 4. 2. 1. 0. 0. 0. 2. 3. 1. 2. 0. 2. 2. 4. 3. 3. 3. 1. 0. 3.
 0. 4. 0. 0. 0. 4. 0. 2.]
